In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import os
import datetime
import pandas as pd
import shutil

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf
import tensorflow.keras.backend as K


from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#from Unet_Resnet_Vgg16_Densenet__v22 import build_double_iaunet, focal_dice_boundary_loss

In [ ]:
mol='_densenet_v5T4'

# Parameters
PATCH_SIZE = 256
NUM_CHANNELS = 3
NUM_CLASSES = 1
BATCH_SIZE = 16

dt_tm=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

sys_info=dt_tm
dt_tm = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
file_Sig = mol + dt_tm

# Update paths for Kaggle
patches_dir = "/content/gdrive/MyDrive/ITMO/01_BRIGHT_FIELD/857x21"

out_path= '/content/gdrive/MyDrive/ITMO/008_OUTPUT/008_Output_Files/'+file_Sig+'/'
tensor_path='/content/gdrive/MyDrive/ITMO/008_OUTPUT/008_TensorBoard/'+file_Sig+'/'
tensor_log_path='/content/gdrive/MyDrive/ITMO/008_OUTPUT/008_TensorBoard/'+file_Sig+'/log/'
keras_path='/content/gdrive/MyDrive/ITMO/008_OUTPUT/008_Keras/'+file_Sig+"/"

keras_file=keras_path+file_Sig+'.keras'

os.makedirs(os.path.join(out_path, 'images'), exist_ok=True)
os.makedirs(keras_path, exist_ok=True)
os.makedirs(tensor_log_path, exist_ok=True)

print(f"File Signature:{file_Sig} indicates file/folder name")

File Signature:_densenet_v5T42025-08-20 15:28:26 indicates file/folder name


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Add, Conv2D

try:
    from tensorflow_addons.layers import InstanceNormalization
    instance_norm_available = True
except ImportError:
    InstanceNormalization = None
    instance_norm_available = False

class InstanceActivation(tf.keras.layers.Layer):
    def __init__(self, name_prefix='ia', debug=False, **kwargs):
        super().__init__(name=name_prefix, **kwargs)
        self.name_prefix = name_prefix
        self.debug = debug

    def build(self, input_shape):
        self.filters = input_shape[-1]
        self.conv = Conv2D(self.filters, (1, 1), activation='relu', padding='same', name=f'{self.name_prefix}_conv')
        self.bn = BatchNormalization(name=f'{self.name_prefix}_bn')

    def call(self, x):
        out = self.conv(x)
        out = self.bn(out)
        if self.debug:
            tf.print(f"[DEBUG] {self.name_prefix}: input shape =", tf.shape(x), ", out shape =", tf.shape(out))
        return Add(name=f'{self.name_prefix}_add')([x, out])

def attention_gate(x, g, inter_channels, name_prefix):
    theta_x = layers.Conv2D(inter_channels, (1, 1), name=f'{name_prefix}_theta_x')(x)
    phi_g = layers.Conv2D(inter_channels, (1, 1), name=f'{name_prefix}_phi_g')(g)
    add = layers.Add(name=f'{name_prefix}_add')([theta_x, phi_g])
    act = layers.Activation('relu', name=f'{name_prefix}_relu')(add)
    psi = layers.Conv2D(1, (1, 1), name=f'{name_prefix}_psi')(act)
    psi = layers.Activation('sigmoid', name=f'{name_prefix}_sigmoid')(psi)
    return layers.Multiply(name=f'{name_prefix}_multiply')([x, psi])

def upsample(filters, size):
    return tf.keras.Sequential([
        layers.Conv2DTranspose(filters, size, strides=2, padding='same'),
        layers.ReLU(),
        BatchNormalization()
    ])

def decoder_conv_block(x, filters, name_prefix):
    x = Conv2D(filters, (3, 3), padding='same', name=f'{name_prefix}_conv1')(x)
    x = BatchNormalization(name=f'{name_prefix}_bn1')(x)
    x = layers.ReLU(name=f'{name_prefix}_relu1')(x)
    x = Conv2D(filters, (3, 3), padding='same', name=f'{name_prefix}_conv2')(x)
    x = BatchNormalization(name=f'{name_prefix}_bn2')(x)
    x = layers.ReLU(name=f'{name_prefix}_relu2')(x)
    return x

def norm_layer(use_instance_norm):
    if use_instance_norm and instance_norm_available:
        return InstanceNormalization()
    return BatchNormalization()

def Unet_Resnet_Vgg16(
    backbone='resnet50',
    input_shape=(256, 256, 3),
    num_classes=1,
    use_attention=False,
    use_instance_norm=False,
    use_ia=False,
    use_mixed_precision=False
):
    if use_mixed_precision:
        from tensorflow.keras import mixed_precision
        mixed_precision.set_global_policy('mixed_float16')

    if backbone == 'resnet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        layer_names = ['conv1_relu', 'conv2_block3_out', 'conv3_block4_out', 'conv4_block6_out', 'conv5_block3_out']
        backbone_filters = [64, 256, 512, 1024, 2048]
    elif backbone == 'vgg16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        layer_names = ['block1_pool', 'block2_pool', 'block3_pool', 'block4_pool', 'block5_pool']
        backbone_filters = [64, 128, 256, 512, 512]
    else:
        raise ValueError("Supported backbones: 'resnet50', 'vgg16'")

    for layer in base_model.layers:
        layer.trainable = False

    inputs = base_model.input
    e1 = base_model.get_layer(layer_names[0]).output
    e2 = base_model.get_layer(layer_names[1]).output
    e3 = base_model.get_layer(layer_names[2]).output
    e4 = base_model.get_layer(layer_names[3]).output
    e5 = base_model.get_layer(layer_names[4]).output

    # Decoder Block 1
    d1 = upsample(backbone_filters[-2], 3)(e5)
    if use_attention:
        e4 = attention_gate(e4, d1, backbone_filters[-2] // 2, name_prefix='attn1')
    d1 = layers.Concatenate()([d1, e4])
    d1 = decoder_conv_block(d1, backbone_filters[-2], name_prefix='dec1')
    if use_ia:
        d1 = InstanceActivation(name_prefix='ia1')(d1)
    d1 = Dropout(0.2)(d1)

    # Decoder Block 2
    d2 = upsample(backbone_filters[-3], 3)(d1)
    if use_attention:
        e3 = attention_gate(e3, d2, backbone_filters[-3] // 2, name_prefix='attn2')
    d2 = layers.Concatenate()([d2, e3])
    d2 = decoder_conv_block(d2, backbone_filters[-3], name_prefix='dec2')
    if use_ia:
        d2 = InstanceActivation(name_prefix='ia2')(d2)
    d2 = Dropout(0.3)(d2)

    # Decoder Block 3
    d3 = upsample(backbone_filters[-4], 3)(d2)
    if use_attention:
        e2 = attention_gate(e2, d3, backbone_filters[-4] // 2, name_prefix='attn3')
    d3 = layers.Concatenate()([d3, e2])
    d3 = decoder_conv_block(d3, backbone_filters[-4], name_prefix='dec3')
    if use_ia:
        d3 = InstanceActivation(name_prefix='ia3')(d3)
    d3 = Dropout(0.35)(d3)

    # Decoder Block 4
    d4 = upsample(backbone_filters[-5], 3)(d3)
    if use_attention:
        e1 = attention_gate(e1, d4, backbone_filters[-5] // 2, name_prefix='attn4')
    d4 = layers.Concatenate()([d4, e1])
    d4 = decoder_conv_block(d4, backbone_filters[-5], name_prefix='dec4')
    if use_ia:
        d4 = InstanceActivation(name_prefix='ia4')(d4)
    d4 = Dropout(0.4)(d4)

    d5 = upsample(backbone_filters[0], 3)(d4)
    d5 = decoder_conv_block(d5, backbone_filters[0], name_prefix='dec5')

    outputs = layers.Conv2D(
        num_classes,
        kernel_size=1,
        activation='sigmoid',
        kernel_regularizer=regularizers.L2(1e-4),
        dtype='float32' if use_mixed_precision else None
    )(d5)

    model = Model(inputs, outputs)
    return model


#  Example usage:
model = Unet_Resnet_Vgg16(
    backbone='vgg16',
    use_attention=True,
    use_ia=False,
    use_instance_norm=False,
    use_mixed_precision=True
)



# Example usage:
model = Unet_Resnet_Vgg16(backbone='vgg16', use_attention=True, use_ia=True)
print("Total params:", model.count_params())
print("Trainable params:", sum(tf.keras.backend.count_params(p) for p in model.trainable_weights))

# Debug print for attention and IA layers
print("\nAttention layers params:")
for layer in model.layers:
    if 'attn' in layer.name:
        print(f"{layer.name}: {layer.count_params()} params")

print("\nInstance Activation layers params:")
for layer in model.layers:
    if 'ia' in layer.name:
        print(f"{layer.name}: {layer.count_params()} params")


I0000 00:00:1754099630.093919      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1754099630.094671      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Total params: 28851173
Trainable params: 14128421

Attention layers params:
attn1_theta_x: 131328 params
attn1_phi_g: 131328 params
attn1_add: 0 params
attn1_relu: 0 params
attn1_psi: 257 params
attn1_sigmoid: 0 params
attn1_multiply: 0 params
attn2_theta_x: 32896 params
attn2_phi_g: 32896 params
attn2_add: 0 params
attn2_relu: 0 params
attn2_psi: 129 params
attn2_sigmoid: 0 params
attn2_multiply: 0 params
attn3_theta_x: 8256 params
attn3_phi_g: 8256 params
attn3_add: 0 params
attn3_relu: 0 params
attn3_psi: 65 params
attn3_sigmoid: 0 params
attn3_multiply: 0 params
attn4_theta_x: 2080 params
attn4_phi_g: 2080 params
attn4_add: 0 params
attn4_relu: 0 params
attn4_psi: 33 params
attn4_sigmoid: 0 params
attn4_multiply: 0 params

Instance Activation layers params:
sequential_5: 2361856 params
ia1: 264704 params
sequential_6: 1180928 params
ia2: 66816 params
sequential_7: 295552 params
ia3: 17024 params
sequential_8: 74048 params
ia4: 4416

In [ ]:
class PatchDataGenerator(Sequence):
    def __init__(self, patches_dir, indices=None, batch_size=BATCH_SIZE, img_size=PATCH_SIZE, **kwargs):  # ✅ Include **kwargs here
        super().__init__(**kwargs)  # ✅ Now this works
        self.batch_size = batch_size
        self.img_size = img_size
        self.image_files = sorted([os.path.join(patches_dir, 'images', fname) for fname in os.listdir(os.path.join(patches_dir, 'images'))])
        self.mask_files = sorted([os.path.join(patches_dir, 'masks', fname) for fname in os.listdir(os.path.join(patches_dir, 'masks'))])
        if indices is not None:
            self.image_files = [self.image_files[i] for i in indices]
            self.mask_files = [self.mask_files[i] for i in indices]
        self.indices = np.arange(len(self.image_files))

    def __len__(self):
        return len(self.image_files) // self.batch_size

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        X_batch = np.array([np.load(self.image_files[i]) for i in batch_indices])
        Y_batch = np.array([np.load(self.mask_files[i]) for i in batch_indices])
        return X_batch, Y_batch

In [ ]:

# Metrics and Loss

def jaccard_score(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    y_pred = K.round(y_pred)  # Convert probabilities to binary predictions (0 or 1)
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred) - intersection
    return (intersection + K.epsilon()) / (union + K.epsilon())  # Add epsilon to avoid division by zero


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def dice_loss(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return 1 - (2. * intersection + K.epsilon()) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())

def dice_bce_loss(y_true, y_pred, bce_weight=0.5, smooth=1e-6):
    # Dice Loss
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    dice_loss = 1 - (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

    # Weighted BCE (no manual weight needed if using class weights)
    bce_loss = K.binary_crossentropy(y_true, y_pred)

    # Combined Loss
    return bce_weight * bce_loss + (1 - bce_weight) * dice_loss
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1. - K.epsilon())
        cross_entropy = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)
        weight = alpha * K.pow(1 - y_pred, gamma) * y_true + (1 - alpha) * K.pow(y_pred, gamma) * (1 - y_true)
        loss = weight * cross_entropy
        return K.mean(loss)
    return focal_loss_fixed

def boundary_loss(y_true, y_pred):
    # Approximate boundary loss using Laplacian filter on y_pred and y_true
    def laplacian_kernel():
        return tf.constant([[0., 1., 0.],
                            [1., -4., 1.],
                            [0., 1., 0.]], shape=(3, 3, 1, 1), dtype=tf.float32)

    kernel = laplacian_kernel()
    y_true_edge = tf.nn.conv2d(y_true, kernel, strides=1, padding='SAME')
    y_pred_edge = tf.nn.conv2d(y_pred, kernel, strides=1, padding='SAME')
    diff = tf.abs(y_true_edge - y_pred_edge)
    return K.mean(diff)

def focal_dice_boundary_loss(y_true, y_pred, alpha=0.25, gamma=2.0, bce_weight=0.3, boundary_weight=0.1):
    fl = focal_loss(gamma=gamma, alpha=alpha)(y_true, y_pred)
    dl = dice_loss(y_true, y_pred)
    bl = boundary_loss(y_true, y_pred)
    return bce_weight * fl + (1 - bce_weight - boundary_weight) * dl + boundary_weight * bl

In [ ]:
model = Unet_Resnet_Vgg16(
    backbone='vgg16',
    use_attention=True,
    use_instance_norm=False,
    use_mixed_precision=True,
    use_ia=False
)



from tensorflow.keras.optimizers.schedules import ExponentialDecay
lr_schedule1 = ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=10000,
    decay_rate=0.9
)


model.compile(
    optimizer=Adam(learning_rate=lr_schedule1),
    loss= focal_dice_boundary_loss, #binary_crossentropy,
    metrics=[
        tf.keras.metrics.TruePositives(name="TP"),
        tf.keras.metrics.FalsePositives(name="FP"),
        tf.keras.metrics.TrueNegatives(name="TN"),
        tf.keras.metrics.FalseNegatives(name="FN"),
        'accuracy',
        recall_m,
        precision_m,
        f1_m,
        jaccard_score
    ]
)
model.summary()

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_12      │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_3 (Cast)       │ (None, 256, 256,  │          0 │ input_layer_12[0… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 256, 256,  │      1,792 │ cast_3[0][0]      │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 256, 256,  │     36,928 │ block1_conv1[0][… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_pool         │ (None, 128, 128,  │          0 │ block1_conv2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv1        │ (None, 128, 128,  │     73,856 │ block1_pool[0][0] │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_conv2        │ (None, 128, 128,  │    147,584 │ block2_conv1[0][… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 64, 64,    │          0 │ block2_conv2[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv1        │ (None, 64, 64,    │    295,168 │ block2_pool[0][0] │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv2        │ (None, 64, 64,    │    590,080 │ block3_conv1[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_conv3        │ (None, 64, 64,    │    590,080 │ block3_conv2[0][… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_pool         │ (None, 32, 32,    │          0 │ block3_conv3[0][… │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv1        │ (None, 32, 32,    │  1,180,160 │ block3_pool[0][0] │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv2        │ (None, 32, 32,    │  2,359,808 │ block4_conv1[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_conv3        │ (None, 32, 32,    │  2,359,808 │ block4_conv2[0][… │
│ (Conv2D)            │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block4_pool         │ (None, 16, 16,    │          0 │ block4_conv3[0][… │
│ (MaxPooling2D)      │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block5_conv1        │ (None, 16, 16,    │  2,359,808 │ block4_pool[0][0

 Total params: 28,498,213 (108.71 MB)

 Trainable params: 13,777,381 (52.56 MB)

 Non-trainable params: 14,720,832 (56.16 MB)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
import os

# Known prefixes
PREFIXES = ["C_", "S_", "H_", "g_", ""]  # "" handles plain img_/mask_

# Helper: extract prefix, key, and type (image/mask)
def extract_key_and_prefix(fname, img_or_mask):
    for prefix in PREFIXES:
        img_prefix = f"{prefix}img_" if prefix else "img_"
        mask_prefix = f"{prefix}mask_" if prefix else "mask_"

        if img_or_mask == "img" and fname.startswith(img_prefix):
            return fname.replace(img_prefix, ""), prefix if prefix else "plain"
        elif img_or_mask == "mask" and fname.startswith(mask_prefix):
            return fname.replace(mask_prefix, ""), prefix if prefix else "plain"
    return None, None  # Unrecognized

# Directories
image_dir = os.path.join(patches_dir, 'images')
mask_dir = os.path.join(patches_dir, 'masks')

# Build image dictionary
image_dict = {}
for fname in os.listdir(image_dir):
    if not fname.endswith(".npy"):
        continue
    key, prefix = extract_key_and_prefix(fname, "img")
    if key:
        image_dict[(key, prefix)] = os.path.join(image_dir, fname)

# Build mask dictionary
mask_dict = {}
for fname in os.listdir(mask_dir):
    if not fname.endswith(".npy"):
        continue
    key, prefix = extract_key_and_prefix(fname, "mask")
    if key:
        mask_dict[(key, prefix)] = os.path.join(mask_dir, fname)

# Get matching image-mask pairs
matching_keys = sorted(set(image_dict.keys()) & set(mask_dict.keys()))

# Final aligned file lists
image_files = [image_dict[k] for k in matching_keys]
mask_files = [mask_dict[k] for k in matching_keys]

In [ ]:
# Data Split
train_indices, val_indices = train_test_split(np.arange(len(image_files)), test_size=0.2, random_state=42)

train_gen = PatchDataGenerator(patches_dir, indices=train_indices, batch_size=BATCH_SIZE)
val_gen = PatchDataGenerator(patches_dir, indices=val_indices, batch_size=BATCH_SIZE)

In [ ]:
# Callbacks
log_dir = os.path.join(tensor_log_path, "fit_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=True, update_freq='epoch' )
#checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(keras_path, save_best_only=True)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=keras_path+file_Sig+"_1st_{epoch:02d}.keras",  # dynamic naming
    save_freq='epoch',
    save_weights_only=False,  # Save the full model (not just weights)
    save_best_only=False,     # Save all epochs (not just the best one)
    verbose=1
)

csv_logger = tf.keras.callbacks.CSVLogger(training_log.csv), append=True)

In [ ]:
# Train with more epochs
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=45,
    callbacks=[csv_logger, tensorboard_callback, checkpoint_callback, early_stopping],
    verbose=1,
    #workers=45,
    #use_multiprocessing=True,
    #max_queue_size=10
)

Epoch 1/45


I0000 00:00:1754052622.739993      95 service.cc:148] XLA service 0x7ea9500039e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754052622.741159      95 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1754052622.741179      95 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1754052625.373793      95 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-01 12:50:48.094542: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f16[16,256,256,64]{3,2,1,0}, u8[0]{0}) custom-call(f16[16,256,256,64]{3,2,1,0}, f16[64,3,3,64]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=b01f_o01i->b01f, custom_call_target="__cudnn$convBackwardInput", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_que

899/899 ━━━━━━━━━━━━━━━━━━━━ 0s 447ms/step - FN: 26600904.0000 - FP: 47943552.0000 - TN: 322202080.0000 - TP: 75112624.0000 - accuracy: 0.8239 - f1_m: 0.6385 - jaccard_score: 0.4784 - loss: 0.3059 - precision_m: 0.5820 - recall_m: 0.7264
Epoch 1: saving model to /kaggle/working/output/_densenet_v5T42025-08-01 12:49:51/keras/_densenet_v5T42025-08-01 12:49:51_1st_01.keras
899/899 ━━━━━━━━━━━━━━━━━━━━ 708s 571ms/step - FN: 26629072.0000 - FP: 47982060.0000 - TN: 322574880.0000 - TP: 75196272.0000 - accuracy: 0.8240 - f1_m: 0.6385 - jaccard_score: 0.4785 - loss: 0.3058 - precision_m: 0.5821 - recall_m: 0.7264 - val_FN: 18170396.0000 - val_FP: 6841729.0000 - val_TN: 180200992.0000 - val_TP: 30716550.0000 - val_accuracy: 0.8940 - val_f1_m: 0.7035 - val_jaccard_score: 0.5468 - val_loss: 0.2307 - val_precision_m: 0.8159 - val_recall_m: 0.6236
Epoch 2/45
899/899 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - FN: 23873320.0000 - FP: 28061994.0000 - TN: 342896256.0000 - TP: 77027608.0000 - accuracy: 0.8891

In [ ]:
import os
import pandas as pd
df = pd.read_csv("training_log.csv")
print(df.head())

   epoch          FN          FP           TN           TP  accuracy  \
0      0  51951056.0  82601328.0  657718912.0  150398384.0  0.857265   
1      1  47208444.0  54913640.0  685406656.0  155140992.0  0.891668   
2      2  42829272.0  47182112.0  693138496.0  159520224.0  0.904514   
3      3  39332256.0  40806928.0  699513024.0  163017056.0  0.914987   
4      4  36422644.0  36741604.0  703579008.0  165926752.0  0.922387   

       f1_m  jaccard_score      loss  precision_m  ...      val_FN     val_FP  \
0  0.691194       0.533192  0.261603     0.650469  ...  18170396.0  6841729.0   
1  0.750017       0.602590  0.208050     0.735015  ...  13140079.0  8910188.0   
2  0.777761       0.638615  0.186267     0.768609  ...  14441355.0  5924799.0   
3  0.800594       0.669569  0.168675     0.796596  ...  12277599.0  7112409.0   
4  0.817353       0.693049  0.155779     0.815864  ...  10190260.0  7596250.0   

        val_TN      val_TP  val_accuracy  val_f1_m  val_jaccard_score  \
0  1802

In [ ]:
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')


drive_folder = tensor_log_path
os.makedirs(drive_folder, exist_ok=True)


# It saves locally in Colab as 'training_log.csv'
local_csv_path = 'training_log.csv'
drive_csv_path = os.path.join(drive_folder, f'training_log_{timestamp}.csv')

if os.path.exists(local_csv_path):
    shutil.copy(local_csv_path, drive_csv_path)
    print(f"Copied existing CSVLogger file to: {drive_csv_path}")

#If we have a DataFrame named `df` (e.g., from TensorBoard logs)
# Save it directly
try:
    df.to_csv(drive_csv_path, index=False)
    print(f"Saved DataFrame to: {drive_csv_path}")
except NameError:
    print("No DataFrame named 'df' found. Skipping direct save.")


NameError: name 'datetime' is not defined

In [ ]:
def visualize_predictions(generator, model, num_images=5, save_dir=out_path):
    os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist

    for i in range(num_images):
        X_batch, Y_batch = generator[i]
        preds = (model.predict(X_batch) > 0.5).astype("int32")

        fig, axs = plt.subplots(1, 3, figsize=(6, 2))

        axs[0].imshow(X_batch[0])
        axs[0].set_title("Original Image")

        axs[1].imshow(Y_batch[0].squeeze(), cmap='gray')
        axs[1].set_title("Ground Truth Mask")

        axs[2].imshow(preds[0].squeeze(), cmap='gray')
        axs[2].set_title("Predicted Mask")

        # Remove axes for clarity
        for ax in axs:
            ax.axis('off')

        # Save figure
        fig_path = os.path.join(save_dir, f"prediction_{i+1}.png")
        plt.savefig(fig_path, bbox_inches='tight')
        #plt.close(fig)

        # Display inline
        plt.show(fig)

        # Optional: close to free memory
        plt.close(fig)
        print(f"Saved: {fig_path}")

# Run Visualization
visualize_predictions(val_gen, model, num_images=40)

NameError: name 'out_path' is not defined

In [ ]:
import os

root_dir = '/kaggle/working'

for root, dirs, files in os.walk(root_dir):
    level = root.replace(root_dir, '').count(os.sep)
    indent = ' ' * 4 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{sub_indent}{f}")

working/
    .virtual_documents/
    tensorboard/
        log/
            fit_20250801-124959/
                validation/
                    events.out.tfevents.1754053197.c9d9347db4da.36.1.v2
                train/
                    events.out.tfevents.1754052600.c9d9347db4da.36.0.v2
    output/
        _densenet_v5T42025-08-01 12:49:51/
            training_log.csv
            keras/
                _densenet_v5T42025-08-01 12:49:51_1st_30.keras
                _densenet_v5T42025-08-01 12:49:51_1st_18.keras
                _densenet_v5T42025-08-01 12:49:51_1st_08.keras
                _densenet_v5T42025-08-01 12:49:51_1st_01.keras
                _densenet_v5T42025-08-01 12:49:51_1st_12.keras
                _densenet_v5T42025-08-01 12:49:51_1st_21.keras
                _densenet_v5T42025-08-01 12:49:51_1st_13.keras
                _densenet_v5T42025-08-01 12:49:51_1st_16.keras
                _densenet_v5T42025-08-01 12:49:51_1st_29.keras
                _densenet_v5T42025-08-0